PyGame Snake

In [3]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\data\HiDrive\Data\uni\dl\code\ml_pygame\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import math
import random as rnd
print(math.sqrt(222))

14.89966442575134


[1, 2]


In [7]:
#https://www.pygame.org/docs/
# Example file showing a circle moving on screen
import pygame

# pygame setup
pygame.init()
screen = pygame.display.set_mode((1280, 720))
clock = pygame.time.Clock()
running = True
dt = 0

player_pos = pygame.Vector2(screen.get_width() / 2, screen.get_height() / 2)
tails=[]
player_dest_y = 100
player_dest_x = 0
random_obj = pygame.Vector2(screen.get_width() / 4, screen.get_height() / 4)
while running:
    
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    dst = math.sqrt( (player_pos.y-random_obj.y) ** 2 + (player_pos.x-random_obj.x) ** 2 )
    if dst < 80:
        rnd.randint(0,screen.get_width())
        random_obj = pygame.Vector2(rnd.randint(0,screen.get_width()), rnd.randint(0,screen.get_height()))   
        tails.append(pygame.Vector2(player_pos.y,player_pos.x))
    # fill the screen with a color to wipe away anything from last frame
    screen.fill("purple")

    pygame.draw.circle(screen, "red", player_pos, 40)
    for t in tails:
        pygame.draw.circle(screen, "red", t, 40)
    pygame.draw.circle(screen, "blue", random_obj, 40)

    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        player_dest_y,player_dest_x = -100,0
    elif keys[pygame.K_s]:
        player_dest_y,player_dest_x = 100,0
    elif keys[pygame.K_a]:
        player_dest_y,player_dest_x = 0,-100
    elif keys[pygame.K_d]:
        player_dest_y,player_dest_x = 0,100

    player_pos.y += player_dest_y* dt
    player_pos.x += player_dest_x * dt    
    for t in tails:
        t.x += player_dest_x * dt   
        t.y += player_dest_y * dt     
    #distance = math.sqrt((player_pos.y - random_obj.y) ** 2 + (player_pos.x - random_obj.x) ** 2)
    #
    # Kollision überprüfen
    #if distance <= 40 + 40:
    #    print("Kollision!")
    # flip() the display to put your work on screen
    pygame.display.flip()

    # limits FPS to 60
    # dt is delta time in seconds since last frame, used for framerate-
    # independent physics.
    dt = clock.tick(60) /1000

pygame.quit()